In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
# Import the requests library.
import requests

# Import the time library.
import time

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the API key.
from config import weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-77.37502783408146 -46.36085990968317
20.17928080990565 -53.05330650170514
-9.531252733198983 113.11291798722107
4.717498861045115 92.87048238453104
86.92309887063658 7.210774945223051
32.35674385693419 -130.59217565246516
-85.2847534164892 64.17684884232327
72.37572177043037 -130.84894042145334
-88.0332591625285 -16.605021379443457
-58.58286991702822 -141.7388857724371
-11.75407058864711 136.13322197955665
33.76504522859511 -70.61496191777077
-0.35509264332159773 -40.552154920921
39.429309973127715 85.55417686756749
-74.03747727183192 108.35555742911032
70.95689965743139 17.51949212421968
15.869087783960367 -178.18270428977394
4.4338333440466755 105.9755331843997
58.67483532757842 157.99307102086738
68.8538317905977 -72.25178757197578
-61.48464365324644 -56.27191559434168
48.29803426169485 -126.00006032497093
80.84587657823707 -78.1908104907972
-44.46698419813649 -10.037285622178302
82.12782664357132 -162.71590067412308
-54.512225490977336 166.79542175364872
-55.8538533184261 -132.919

-49.76110115596234 80.49299923752619
-31.088517877636143 136.99873188563515
-7.323672978458632 -46.48888389230467
-58.06942737031073 75.85525328818579
-30.71923995476297 -101.94941708506455
29.80384497102129 90.8716037543582
52.95322302227527 -118.95924056607005
80.31798936269763 35.82269557648431
-74.93616326981697 22.281359934224156
-38.312059911048614 30.89315842152908
-31.349429137642076 96.04977467021848
77.64723876640113 100.03660951923786
-29.863896820590107 101.51507147097328
-25.41236277247026 96.59910748548617
-26.077978984736852 -97.75376120222444
15.261033606463045 -158.93670376461245
68.91698010549834 -88.77182481645647
-61.43927424677901 -58.887874354406804
55.90921073616087 -27.371628041760943
65.84493593167079 -132.39877991977005
-17.732360650494826 17.677579062475758
-72.59253721897068 -154.1532849491696
65.66044646683201 -11.758509536256724
-74.48862674580391 -38.50697065793821
-30.33929024666392 -79.0393069742866
-6.809913509643479 -22.840287886248632
19.150177987357

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

765

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=6c95cf4ddc6e38b59502909f11fae650


In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | saint-francois
Processing Record 3 of Set 1 | ambulu
Processing Record 4 of Set 1 | banda aceh
Processing Record 5 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 6 of Set 1 | half moon bay
Processing Record 7 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 8 of Set 1 | tuktoyaktuk
Processing Record 9 of Set 1 | mataura
Processing Record 10 of Set 1 | galiwinku
City not found. Skipping...
Processing Record 11 of Set 1 | hamilton
Processing Record 12 of Set 1 | camocim
Processing Record 13 of Set 1 | korla
Processing Record 14 of Set 1 | albany
Processing Record 15 of Set 1 | finnsnes
Processing Record 16 of Set 1 | kapaa
Processing Record 17 of Set 1 | kerteh
City not found. Skipping...
Processing Record 18 of Set 1 | tigil
Processing Record 19 of Set 1 | clyde river
Processing Record 20 of Set 1 | ucluelet
Processing 

Processing Record 38 of Set 4 | karakendzha
City not found. Skipping...
Processing Record 39 of Set 4 | kenai
Processing Record 40 of Set 4 | rocha
Processing Record 41 of Set 4 | nizhneyansk
City not found. Skipping...
Processing Record 42 of Set 4 | moree
Processing Record 43 of Set 4 | bilibino
Processing Record 44 of Set 4 | bubaque
Processing Record 45 of Set 4 | klaksvik
Processing Record 46 of Set 4 | santa maria
Processing Record 47 of Set 4 | chara
Processing Record 48 of Set 4 | christchurch
Processing Record 49 of Set 4 | los llanos de aridane
Processing Record 50 of Set 4 | hithadhoo
Processing Record 1 of Set 5 | vikhorevka
Processing Record 2 of Set 5 | naze
Processing Record 3 of Set 5 | nikolskoye
Processing Record 4 of Set 5 | benjamin hill
Processing Record 5 of Set 5 | sorland
Processing Record 6 of Set 5 | manado
Processing Record 7 of Set 5 | tyup
Processing Record 8 of Set 5 | os
City not found. Skipping...
Processing Record 9 of Set 5 | sobolevo
Processing Record

Processing Record 18 of Set 8 | babanusah
City not found. Skipping...
Processing Record 19 of Set 8 | porto santo
Processing Record 20 of Set 8 | yambio
Processing Record 21 of Set 8 | mouila
Processing Record 22 of Set 8 | kedrovyy
Processing Record 23 of Set 8 | svetlaya
Processing Record 24 of Set 8 | homer
Processing Record 25 of Set 8 | katherine
Processing Record 26 of Set 8 | gold coast
Processing Record 27 of Set 8 | ukiah
Processing Record 28 of Set 8 | laviana
Processing Record 29 of Set 8 | urumqi
Processing Record 30 of Set 8 | northam
Processing Record 31 of Set 8 | manati
Processing Record 32 of Set 8 | hami
Processing Record 33 of Set 8 | itupiranga
Processing Record 34 of Set 8 | oktyabrskoye
Processing Record 35 of Set 8 | aleksandrovsk-sakhalinskiy
Processing Record 36 of Set 8 | tommot
Processing Record 37 of Set 8 | vila
Processing Record 38 of Set 8 | torbay
Processing Record 39 of Set 8 | linda
Processing Record 40 of Set 8 | san vicente
Processing Record 41 of Se

Processing Record 5 of Set 12 | sharjah
Processing Record 6 of Set 12 | sholokhovskiy
Processing Record 7 of Set 12 | husavik
Processing Record 8 of Set 12 | lac du bonnet
Processing Record 9 of Set 12 | gurgan
City not found. Skipping...
Processing Record 10 of Set 12 | shihezi
Processing Record 11 of Set 12 | aitape
Processing Record 12 of Set 12 | antalaha
Processing Record 13 of Set 12 | takoradi
Processing Record 14 of Set 12 | azul
Processing Record 15 of Set 12 | aswan
Processing Record 16 of Set 12 | nizwa
Processing Record 17 of Set 12 | vanavara
Processing Record 18 of Set 12 | ginda
Processing Record 19 of Set 12 | talaya
Processing Record 20 of Set 12 | tambura
Processing Record 21 of Set 12 | tokur
Processing Record 22 of Set 12 | cullinan
Processing Record 23 of Set 12 | lalomanu
City not found. Skipping...
Processing Record 24 of Set 12 | axim
Processing Record 25 of Set 12 | clinton
Processing Record 26 of Set 12 | borlange
City not found. Skipping...
Processing Record 

Processing Record 41 of Set 15 | saint-joseph
Processing Record 42 of Set 15 | brokopondo
Processing Record 43 of Set 15 | svetlogorsk
Processing Record 44 of Set 15 | chicama
Processing Record 45 of Set 15 | nabire
Processing Record 46 of Set 15 | price
Processing Record 47 of Set 15 | pangnirtung
Processing Record 48 of Set 15 | mission
Processing Record 49 of Set 15 | stornoway
Processing Record 50 of Set 15 | xuddur
Processing Record 1 of Set 16 | okhotsk
Processing Record 2 of Set 16 | mecca
Processing Record 3 of Set 16 | ca mau
Processing Record 4 of Set 16 | amli
Processing Record 5 of Set 16 | bestobe
Processing Record 6 of Set 16 | maraba
Processing Record 7 of Set 16 | gazanjyk
Processing Record 8 of Set 16 | bur gabo
City not found. Skipping...
Processing Record 9 of Set 16 | bargal
City not found. Skipping...
Processing Record 10 of Set 16 | margate
Processing Record 11 of Set 16 | kuvshinovo
Processing Record 12 of Set 16 | ulladulla
Processing Record 13 of Set 16 | biak


In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,46.06,scattered clouds,66,40,5.75,AR,2022-08-31 22:22:14
1,Saint-Francois,46.4154,3.9054,63.84,clear sky,82,0,2.30,FR,2022-08-31 22:22:14
2,Ambulu,-8.3450,113.6058,72.61,overcast clouds,93,94,3.22,ID,2022-08-31 22:22:14
3,Banda Aceh,5.5577,95.3222,78.69,broken clouds,81,84,9.82,ID,2022-08-31 22:22:15
4,Half Moon Bay,37.4636,-122.4286,88.57,clear sky,60,0,19.57,US,2022-08-31 22:22:15
5,Tuktoyaktuk,69.4541,-133.0374,64.40,scattered clouds,59,40,8.05,CA,2022-08-31 22:22:15
6,Mataura,-46.1927,168.8643,57.47,broken clouds,71,59,2.82,NZ,2022-08-31 22:22:16
7,Hamilton,39.1834,-84.5333,83.25,scattered clouds,50,40,10.36,US,2022-08-31 22:18:59
8,Camocim,-2.9022,-40.8411,77.76,clear sky,75,2,12.95,BR,2022-08-31 22:22:16
9,Korla,41.7597,86.1469,70.86,clear sky,37,3,13.04,CN,2022-08-31 22:22:16


In [10]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")